# Matrix Multiplication parallelization example 

In [1]:
%%writefile pool_example.py
import numpy as np
import time
from random import random
import concurrent.futures
from functools import partial

def explicit_matmul(A,B):
    #A[m][n]
    #B[n][p]
    #C[m][p]    
    C_temp = np.zeros((np.shape(A)[0],np.shape(A)[1]))
    for i in range(np.shape(A)[0]): #(i=1...m) Rows in A
        for j in range(np.shape(B)[1]): # (j=1...p) Columns in B
            for k in range(np.shape(A)[1]): # (k=1...n) Columns in A
                C_temp[i][j] += A[i][k] * B[k][j]
    return(C_temp)

def explicit_matmul_process(A,B,k):
    #A[m][n]
    #B[n][p]
    #C[m][p]
    C_temp = np.zeros((np.shape(A)[0],np.shape(A)[1]))
    for i in range(np.shape(A)[0]): #(i=1...m) Rows in A
        for j in range(np.shape(B)[1]): # (j=1...p) Columns in B
            #for k in range(np.shape(A)[1]): # (k=1...n) Columns in A
            C_temp[i][j] += A[i][k] * B[k][j]
    return(C_temp)
 
if __name__ == '__main__':
  
     
    AX=AY=BX=BY=300
    print("Multiplying 2 matricies of shape (" +str(AX)+","+str(AY)+")")
    
    A = np.random.rand(AX,AY)
    B = np.random.rand(BX,BY) 

    print("Starting procs")
    start_procs = time.perf_counter()
    
    ##################################################################
    ##################################################################
    ##################################################################
    ## this specific example is trying to get you to use the concurrent.futures.ProcessPoolExecutor.map method
    
    #1: Create an iterable/list for the workers (processes) to work on
    
    C_locations = [i for i in range(AX)]

    #2: Use the partial method from functools to create a "new" function with only 1 Iterable argument!
    ###   Partial allow us to fix a certain number of arguments of a function and generate a new function.
    ###   WE need this because pool.map() only excepts a function and an iterable argument for that function
                
    new_function = partial(explicit_matmul_process,A,B)    

    #3: Create an executor
    nprocs = 3
    print(nprocs)
    executor = concurrent.futures.ProcessPoolExecutor(max_workers=nprocs)

    #4: start the pool of processes
    results = executor.map(new_function,C_locations)
                           
    #5: Make sure results are put back to gether and correct!!!
    ### this is the more difficult step
    ### HINT: use np.sum (Naive)!                          
    C_parallel = np.sum(list(results),axis=0)
    
    ##################################################################
    ##################################################################
    ##################################################################
    
    end_procs = time.perf_counter()
    print("Procs mult: ",end_procs -start_procs)
 
    start_explicit = time.perf_counter()
    C_explicit = explicit_matmul(A,B)
    end_explicit  = time.perf_counter()
    print("Explicit mult: ",end_explicit - start_explicit)
     
    start_np = time.perf_counter()
    C_np = np.matmul(A,B)
    end_np  = time.perf_counter()
    print("NumPy  matmul: ",end_np -start_np)
     
    if not np.allclose(C_explicit, C_parallel, rtol=1e-10, atol=1e-10):
        print("C_parallel is not equal to C_explicit!!")
    if not np.allclose(C_parallel, C_np, rtol=1e-10, atol=1e-10):
        print("C_parallel is not equal to C_np!!")
    if not np.allclose(C_explicit, C_np, rtol=1e-10, atol=1e-10):
        print("C_np is not equal to C_explicit!!")

Overwriting pool_example.py


In [47]:
!python pool_example.py

Multiplying 2 matricies of shape (300,300)
Starting procs
3
Procs mult:  7.860651847004192
Explicit mult:  19.734997980995104
NumPy  matmul:  0.0028645599959418178


### Submit a job to the batch system if you want to use more that 3 cores!

In [33]:
%%writefile python_job.sh
#!/bin/bash
#SBATCH -n 16
#SBATCH -p normal
#SBATCH -t 00:30:00

module load 2021 SciPy-bundle/2021.05-foss-2021a

python pool_example.py



Overwriting python_job.sh


In [30]:
!sbatch python_job.sh

Submitted batch job 10289478
